In [ ]:
import gradio as gr
import json

# Load your model evaluation results
with open("model_evaluation_results.json", "r", encoding="utf-8") as f:
    data = json.load(f)

feedback_data = []

def show_example(i):
    """Return formatted text + current index"""
    item = data[i]
    text = f"""### Query {i+1}/{len(data)}

**Query:** {item['query']}

**Mistral Output:** {item['mistral_output']}

**Gemma Output:** {item['gemma_output']}

**Similarity:** {item['similarity_score']} | **Rating:** {item['rating']}
**Reason:** {item['reason']}"""
    return text, i

def save_feedback(i, feedback):
    """Save feedback and move to next example"""
    entry = data[i].copy()
    entry["human_feedback"] = feedback
    feedback_data.append(entry)

    # Save progress
    with open("human_feedback.json", "w", encoding="utf-8") as f:
        json.dump(feedback_data, f, ensure_ascii=False, indent=2)

    # Move to next item
    next_i = (i + 1) % len(data)
    return *show_example(next_i), ""

with gr.Blocks() as demo:
    gr.Markdown("# Human Feedback on Triple Extraction")

    index_state = gr.State(0)
    display_box = gr.Markdown()
    feedback_box = gr.Textbox(label="Your Feedback", placeholder="Write your thoughts here...", lines=4)
    next_button = gr.Button("Submit & Next")

    # Initialize first example
    init_text, _ = show_example(0)
    display_box.value = init_text

    next_button.click(
        save_feedback,
        inputs=[index_state, feedback_box],
        outputs=[display_box, index_state, feedback_box]
    )

demo.launch()
